In [ ]:
import holodeck as holo
from holodeck import single_sources, utils, plot, detstats
from holodeck.constants import YR

import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import matplotlib.cm as cm


import hasasia.sensitivity as hsen
import hasasia.sim as hsim
import hasasia.skymap as hsky

# Load Strain Library

In [ ]:
# sam02 has r3, f40, 15 yr dur, 40 freqs, ca
hcfile = np.load('/Users/emigardiner/GWs/holodeck/ecg-notebooks/detstats_functions/npz_arrays/sam02_r3_f40_l10000.npz')
dur = hcfile['dur']
cad = hcfile['cad']
fobs = hcfile['fobs'][:]
df_orb = hcfile['df_orb']
hc_ss = hcfile['hc_ss']
hc_bg = hcfile['hc_bg']
hcfile.close()

shape = hc_ss.shape
F, R, L = shape[0], shape[1], shape[2]
print('F, R, L =', F,R,L)
sam_name = 'sam02: dur=%.2f yr, cad=%.4f yr, %d loudest' % (dur/YR, cad/YR, L)
print(sam_name)

# Function to Build Skymap

In [ ]:
NSIDE = 24
print( "Approximate resolution at NSIDE {} is {:.2} deg".format(
        NSIDE, hp.nside2resol(NSIDE, arcmin=True) / 60))
def hc_map(hc_ss, hc_bg, nside=NSIDE):
    """ Build mollview array of strains for a healpix map
    
    Parameters
    ----------
    hc_ss : (F,R,L) NDarray
        Characteristic strain of single sources.
    hc_bg : (F,R) NDarray
        Characteristic strain of the background.
    nside : integer
        number of sides for healpix map.

    Returns
    -------
    moll_hc : (NPIX,) 1Darray
        Array of strain at every pixel for a mollview healpix map.
    
    NOTE: Could speed up the for-loops, but it's ok for now.
    """

    npix = hp.nside2npix(nside)
    nfreqs = len(hc_ss)
    nreals = len(hc_ss[0])
    nloudest = len(hc_ss[0,0])

    # spread background evenly across pixels in moll_hc
    moll_hc = np.ones((nfreqs,nreals,npix)) * hc_bg[:,:,np.newaxis]/np.sqrt(npix) # (frequency, realization, pixel)

    # choose random pixels to place the single sources
    pix_ss = np.random.randint(0, npix-1, size=nfreqs*nreals*nloudest).reshape(nfreqs, nreals, nloudest)
    for ff in range(nfreqs):
        for rr in range(nreals):
            for ll in range(nloudest):
                moll_hc[ff,rr,pix_ss[ff,rr,ll]] = np.sqrt(moll_hc[ff,rr,pix_ss[ff,rr,ll]]**2
                                                          + hc_ss[ff,rr,ll]**2)
                
    return moll_hc



In [ ]:
moll_hc = hc_map(hc_ss, hc_bg)
print(moll_hc.shape)

In [ ]:
# def plot_all_freqs(fobs, moll_hc, nrows, ncols, saveloc=None):
#     """" Map all the freqs for a single realization
#     Parameters
#     ----------
#     fobs : (F,) 1Darray
#         Observed gw frequencies.
#     moll_hc : (F,L) NDarray
#         strains of each pixel for a single realization
#     nrows * ncols should be >= nfreqs
#     """
#     fig, axs = plt.subplots(nrows=nrows, ncols=ncols)
#     for ff in range(F):
#         title='$f$= %.2f yr$^{-1}$ (bin %d/%d)' % (fobs[ff]*YR, ff, F)
#         row, col = int(ff/4), ff%4
#         # print(row,col)
#         plt.axes(axs[row,col])
#         hp.mollview(moll_hc[ff], title=title, hold=True)

#     fig.suptitle('%d Loudest Single Sources' % L, y=.895)
#     fig.tight_layout()
#     fig.set_size_inches(18, 30)
#     if saveloc is not None:
#         fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# # fig


# Calculate Harmonics

In [ ]:
LMAX = 8
def sph_harm_from_map(moll_hc, lmax=LMAX):
    """ Calculate spherical harmonics from strains at every pixel of 
    a healpix mollview map.
    
    Parameters
    ----------
    moll_hc : (NPIX,) 1Darray
        Characteristic strain of each pixel of a healpix map.
    lmax : int
        Highest harmonic to calculate.

    Returns
    -------
    Cl : (F,R,lmax+1) NDarray
        Spherical harmonic coefficients 
        
    """
    nfreqs = len(moll_hc)
    nreals = len(moll_hc[0])

    Cl = np.zeros((nfreqs, nreals, lmax+1))
    for ff in range(nfreqs):
        for rr in range(nreals):
            Cl[ff,rr,:] = hp.anafast(moll_hc[ff,rr], lmax=lmax)

    return Cl

def sph_harm_from_hc(hc_ss, hc_bg, nside = NSIDE, lmax = LMAX):
    """ Calculate spherical harmonics and strain at every pixel
    of a healpix mollview map from single source and background char strains.

    Parameters
    ----------
    hc_ss : (F,R,L) NDarray
        Characteristic strain of single sources.
    hc_bg : (F,R) NDarray
        Characteristic strain of the background.
    nside : integer
        number of sides for healpix map.

    Returns
    -------
    moll_hc : (NPIX,) 1Darray
        Array of strain at every pixel for a mollview healpix map.
    Cl : (F,R,lmax+1) NDarray
        Spherical harmonic coefficients 
    
    """
    moll_hc = hc_map(hc_ss, hc_bg, nside)
    Cl = sph_harm_from_map(moll_hc, lmax)

    return moll_hc, Cl


In [ ]:
lmax = LMAX
Cl = sph_harm_from_map(moll_hc, lmax)
print(Cl.shape)

In [ ]:
moll_hc, Cl = sph_harm_from_hc(hc_ss, hc_bg)

# Plots

In [ ]:
lvals = np.arange(lmax+1)
fobs_nHz = fobs*10**9

In [ ]:
def plot_Cl(Cl, lvals, fobs_nHz):
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear')
    colors=cm.rainbow(np.linspace(0,1,len(Cl)))

    for ff in range(len(Cl)):
        if ff in (0, 10, 20, 30, 39):
            label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
        else: label = None
        ax.plot(lvals[:], Cl[ff,:], c=colors[ff], alpha=0.5, label=label,
                marker='o')
        
    ax.legend()
    return fig

fig = plot_Cl(Cl[:,0,:], lvals, fobs_nHz) # pick one realization

In [ ]:
def plot_ClC0(Cl, lvals, fobs_nHz):
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell/C_0$', xscale='linear')
    colors=cm.rainbow(np.linspace(0,1,len(Cl)))

    for ff in range(len(Cl)):
        if ff in (0, 10, 20, 30, 39):
            label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
        else: label = None
        ax.plot(lvals[:], Cl[ff,:]/Cl[ff,0,np.newaxis], c=colors[ff], alpha=0.5, label=label,
                marker='o')
        
    ax.legend()
    return fig

fig = plot_ClC0(Cl[:,0,:], lvals, fobs_nHz) # pick one realization

In [ ]:
def plot_llp1Cl(Cl, lvals, fobs_nHz):
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$\ell(\ell+1)C_\ell$', xscale='linear')
    colors=cm.rainbow(np.linspace(0,1,len(Cl)))

    xx = lvals[1:]
    yy = (lvals*(lvals+1))[np.newaxis,1:]*Cl[:,1:]
    for ff in range(len(Cl)):
        if ff in (0, 10, 20, 30, 39):
            label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
        else: label = None
        ax.plot(xx, yy[ff], c=colors[ff], alpha=0.5, label=label,
                marker='o')
        
    ax.legend()
    return fig

fig = plot_llp1Cl(Cl[:,0,:], lvals, fobs_nHz) # pick one realization

In [ ]:
def plot_Clg0(Cl, lvals, fobs):
    # can do several realizations

    fig, ax = plot.figax(figsize=(8,5), xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel='$C_{\ell>0}/C_0$', xscale='linear')
    xx = fobs*YR
    print(xx.shape)
    print(np.sum(Cl[:,:,1:], axis=2).shape)
    print(Cl[:,:,0,np.newaxis].shape)
    yy = np.sum(Cl[:,:,1:], axis=2)/Cl[:,:,0]
    print(yy.shape)
    for rr in range(len(Cl[0])):
        ax.plot(xx, yy[:,rr], alpha=0.5, marker='o')

    ax.plot(xx,np.median(yy, axis=1), color='k')
        
    ax.legend()
    return fig

fig = plot_Clg0(Cl, lvals, fobs) # pick one realization

# Rebuild Harmonics

In [ ]:
def map_from_sph_harm(Cl, nside=NSIDE, lmax=LMAX):
    nfreqs = len(Cl)
    nreals = len(Cl[0])
    npix = hp.nside2npix(nside)
    moll_sh = np.zeros((nfreqs,nreals,npix))
    for ff in range(nfreqs):
        for rr in range(nreals):
            moll_sh[ff,rr,:] = hp.synfast(Cl[ff,rr], nside, lmax=lmax)
    return moll_sh


In [ ]:
moll_sh = map_from_sph_harm(Cl)


rr=0
fig, axs = plt.subplots(nrows=10, ncols=int(F/10))
for ff in range(F):
    title='$f$= %.2f yr$^{-1}$ (bin %d/%d)' % (fobs[ff]*YR, ff, F)
    row, col = int(ff/4), ff%4
    # print(row,col)
    plt.axes(axs[row,col])
    hp.mollview(moll_sh[ff,rr], title=title, hold=True)

fig.suptitle('%d Loudest Single Sources, Recovered from Spherical Harmonics' % L, y=.895)
fig.tight_layout()
fig.set_size_inches(18, 30)
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

### Original maps

In [ ]:
rr=0
fig, axs = plt.subplots(nrows=10, ncols=int(F/10))
for ff in range(F):
    title='$f$= %.2f yr$^{-1}$ (bin %d/%d)' % (fobs[ff]*YR, ff, F)
    row, col = int(ff/4), ff%4
    # print(row,col)
    plt.axes(axs[row,col])
    hp.mollview(moll_hc[ff,rr], title=title, hold=True)

fig.suptitle('%d Loudest Single Sources' % L, y=.895)
fig.tight_layout()
fig.set_size_inches(18, 30)
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig